In [ ]:
from __future__ import division
import tensorflow as tf
import pandas as pd
import numpy as np
from  Preprocesscopy import Preprocesscopy

In [ ]:
file_path = '/home/rui/Documents/MLProject/data/modified-crab/trips-in-sanfran.csv'
city = 'san_francisco'
tile_size = 0.360
la_size = 0.003234
long_size = 0.004049
start_id = 0
freq_threshold = 0

In [ ]:
data = Preprocesscopy(file_path, city, tile_size, la_size, long_size, freq_threshold)

In [ ]:
data.read_dataset()
print "done read dataset"
data.split_train_test()
print "done split dataset"
data.find_significant_region(start_id)
print "done find significant region"
data.filter_low_frequency_trajectory()
print "done filter low frequency traj"
data.refind_significant_region(start_id)
print "done refind significant region"
data.convert_train_traj_to_region_id()
print "done convert train traj to region id"
data.convert_test_traj_to_region_id()
print "done convert test traj to region id"

In [ ]:
train_data = data.taxi_train
test_data = data.taxi_test

In [ ]:
len(data.region_dict)

In [ ]:
# step 3: get trip trace and pad.
context_window = 2
train_X = []
train_y = []

for row_index, row in train_data.iterrows():
    single_trip = row[3]
    trip_length = len(single_trip)

    for index in xrange(trip_length):
        if index <= context_window-1:
            continue
        train_X.append(single_trip[index - context_window:index])
        train_y.append(single_trip[index])

    if row_index % 10000 == 0:
        print row_index

train_X = np.array(train_X)
train_y = np.array(train_y)
print train_X.shape

print 'finish extract data'

In [ ]:
# step 3: get trip trace and pad.
test_X = []
test_y = []

for row_index, row in test_data.iterrows():
    single_trip = row[3]
    trip_length = len(single_trip)

    for index in xrange(trip_length):
        if index <= context_window-1:
            continue
        test_X.append(single_trip[index - context_window:index])
        test_y.append(single_trip[index])

    if row_index % 10000 == 0:
        print row_index

test_X = np.array(test_X)
test_y = np.array(test_y)
print test_X.shape

print 'finish extract data'

In [ ]:
# step 4: build and train a simple feedforward neural network model
batch_size = 32
vocabulary_size = len(data.region_dict)
embedding_size = 50
hidden_size = 75
learning_rate = 0.0001

In [ ]:
def data_iterator():
    """ A simple data iterator """
    data_len = len(train_X)

    idxs = np.arange(0, data_len)
    np.random.shuffle(idxs)
    shuf_features = train_X[idxs]
    shuf_labels = train_y[idxs]
    batch_len = data_len // batch_size

    while True:
        for i in range(batch_len):
            batch_train_X = shuf_features[batch_size*i: batch_size*(i+1)]
            batch_train_y = shuf_labels[batch_size*i: batch_size*(i+1)]
            yield batch_train_X, batch_train_y


iter_ = data_iterator()

In [ ]:
def test_data_iterator():
    """ A simple data iterator """
    data_len = len(test_X)
    batch_len = data_len // batch_size

    while True:
        for i in range(batch_len):
            batch_test_X = test_X[batch_size*i: batch_size*(i+1)]
            batch_test_y = test_y[batch_size*i: batch_size*(i+1)]
            yield batch_test_X, batch_test_y


test_iter_ = test_data_iterator()

In [ ]:
# step 6: define the model and train
with tf.Graph().as_default(), tf.Session() as sess:
    inputs_data = tf.placeholder(tf.int32, [batch_size, context_window])
    outputs_data = tf.placeholder(tf.int32, [batch_size])

    projection_weight = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size]))

    hidden_weight_1 = tf.Variable(tf.random_uniform([embedding_size, hidden_size]))
    hidden_bias_1 = tf.Variable(tf.random_uniform([hidden_size]))
    
    hidden_weight_2 = tf.Variable(tf.random_uniform([embedding_size, hidden_size]))
    hidden_bias_2 = tf.Variable(tf.random_uniform([hidden_size]))

    output_weight = tf.Variable(tf.random_uniform([2*hidden_size, embedding_size]))
    output_bias = tf.Variable(tf.random_uniform([embedding_size]))
    
    softmax_weight = tf.Variable(tf.random_uniform([embedding_size, vocabulary_size]))

    projection_layer_1 = tf.nn.embedding_lookup(projection_weight, inputs_data[:, 0])
    hidden_layer_1 = tf.nn.relu(tf.matmul(projection_layer_1, hidden_weight_1) + hidden_bias_1)
    
    projection_layer_2 = tf.nn.embedding_lookup(projection_weight, inputs_data[:, 1])
    hidden_layer_2 = tf.nn.relu(tf.matmul(projection_layer_2, hidden_weight_2) + hidden_bias_2)

    hidden_layer = tf.concat(1, [hidden_layer_1, hidden_layer_2])

    output_layer = tf.nn.relu(tf.matmul(hidden_layer, output_weight) + output_bias)
    
    logits = tf.matmul(output_layer, softmax_weight)
    
    #logits = tf.matmul(output_layer, tf.transpose(projection_weight))

    true_location = outputs_data

    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=true_location)

    mean_loss = tf.reduce_mean(cross_entropy)
    total_loss = tf.reduce_sum(cross_entropy)

    prediction_accuracy = tf.cast(tf.equal(tf.argmax(logits, 1), tf.cast(true_location, tf.int64)), tf.float32)
    mean_accuracy = tf.reduce_mean(prediction_accuracy)
    total_correct = tf.reduce_sum(prediction_accuracy)

    # training step
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(mean_loss)

    # add variable initializer
    init = tf.initialize_all_variables()

    # step 6: begin training
    sess.run(init)
    num_epoch = 1000
    batch_len = len(train_X) // batch_size
    print "begin training the rnn"
    for epoch in range(num_epoch):

        for _ in range(batch_len):

            # get a batch of data
            x_batch, y_batch = iter_.next()

            # pass it in as through feed_dict
            _, loss_val, accuracy_val = sess.run([train_step, mean_loss, mean_accuracy], feed_dict={
                            inputs_data: x_batch,
                            outputs_data: y_batch
                            })

        training_total_num_prediction = 0
        training_total_loss_val = 0
        training_total_num_correct = 0
        for _ in range(batch_len):
            # get a batch of data
            x_batch, y_batch = iter_.next()

            num_pred = x_batch.shape[0]

            # pass it in as through feed_dict
            loss_val, num_correct = sess.run([total_loss, total_correct], feed_dict={
                            inputs_data: x_batch,
                            outputs_data: y_batch
                            })

            training_total_num_prediction += num_pred
            training_total_loss_val += loss_val
            training_total_num_correct += num_correct
        training_overall_mean_loss = training_total_loss_val / training_total_num_prediction
        training_overall_accuracy = training_total_num_correct / training_total_num_prediction
        
        if epoch % 20 == 0:
            test_total_num_prediction = 0
            test_total_loss_val = 0
            test_total_num_correct = 0
            for _ in range(batch_len):
                # get a batch of data
                x_batch, y_batch = test_iter_.next()

                num_pred = x_batch.shape[0]

                # pass it in as through feed_dict
                loss_val, num_correct = sess.run([total_loss, total_correct], feed_dict={
                                inputs_data: x_batch,
                                outputs_data: y_batch
                                })

                test_total_num_prediction += num_pred
                test_total_loss_val += loss_val
                test_total_num_correct += num_correct
            test_overall_mean_loss = test_total_loss_val / test_total_num_prediction
            test_overall_accuracy = test_total_num_correct / test_total_num_prediction
            print "new test accuracy"
            
        print "Epoch %d The loss is %f, the training accuracy is %f, the test accuracy is %f" % (epoch, training_overall_mean_loss, 
                                                                                                 training_overall_accuracy, test_overall_accuracy)